- 출처: https://wikidocs.net/book/14285

## 2.2. 시스템 인스트럭션 사용하기

### 2.2.1. 페르소나 만들기

In [1]:
import google.generativeai as genai
import os

system_instruction = "당신은 초등학교 선생님입니다. 사용자는 초등학교 5학년 학생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f"[사용자]: {user_query}")
    response = chat_session.send_message(user_query)
    print(f"[모델]: {response.text}")

[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 컴퓨터가 마치 사람처럼 생각하고 배울 수 있도록 만든 기술이야.  영화에서 보는 로봇처럼 말하는 것도 있고, 네가 쓰는 검색엔진도 인공지능을 이용한 거란다.  우리가 더 편리하게 살 수 있도록 도와주는 아주 똑똑한 기술이지!

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 인공지능은 스스로 생각하는 것처럼 보이지만,  사람이 미리 입력해 준 정보를 바탕으로  판단하고 행동하는 거야. 마치 복잡한 규칙을 가진 장난감 같지!



### 2.2.2. 답변 형식 지정하기

In [2]:
import json
system_instruction='JSON schema로 주제별로 답하되 3개를 넘기지 말 것:{{"주제": <주제>, "답변":<두 문장 이내>}}'
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=system_instruction, generation_config={"response_mime_type": "application/json"})
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능의 특징이 뭐에요?", "어떤 것들을 조심해야 하죠?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    answer_dict = json.loads(response.text)
    print(answer_dict)

[사용자]: 인공지능의 특징이 뭐에요?
{'주제': '인공지능의 특징', '답변': '인공지능은 학습, 추론, 자가개선을 통해 인간의 지능적 행동을 모방합니다.  방대한 데이터를 분석하고 패턴을 인식하여 복잡한 문제를 해결하는 능력이 특징입니다.'}
[사용자]: 어떤 것들을 조심해야 하죠?
{'주제': '인공지능의 위험성', '답변': '인공지능의 편향된 데이터 학습으로 인한 차별 및 오류 가능성과 프라이버시 침해, 일자리 감소 등의 부정적 영향에 유의해야 합니다.'}


### 2.2.3. 구조화된 출력(Structured output) 사용하기

In [ ]:
json_schema = {
    'properties': {
        'product_name': {
            'type': 'string'
        },
        'size': {
            'enum': ['S', 'M', 'L', 'XL'],
            'type': 'string'
        },
        'price': {
            'type': 'integer'
        },
    },
    'required': ['price', 'size', 'product_name'],
    'type': 'object'
}

model = genai.GenerativeModel("gemini-1.5-flash-latest")
response = model.generate_content(
    "눈이오는 겨울 등산을 좋아하는 남성의 옷을 추천해주세요",
    generation_config=genai.GenerationConfig( 
        response_mime_type="application/json", response_schema=json_schema
    ),
)
print(response.text)

{"price": 150, "product_name": "겨울 등산용 남성 패딩 점퍼", "size": "L"}


## 2.3. 입력 구조 살펴보기

### 2.3.1. Content 객체

In [4]:
import google.generativeai as genai
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.text)

인공지능(AI)은 인간의 지능을 모방하여 학습, 문제 해결, 의사 결정과 같은 지능적인 작업을 수행하는 컴퓨터 시스템입니다.



In [7]:
response.candidates[0].content

parts {
  text: "인공지능(AI)은 인간의 지능을 모방하여 학습, 문제 해결, 의사 결정과 같은 지능적인 작업을 수행하는 컴퓨터 시스템입니다.\n"
}
role: "model"

In [8]:
#multi_turn1.py
chat_session = model.start_chat(history=[]) #ChatSession 객체 반환
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]
for user_query in user_queries:
    print(f'[사용자]: {user_query}')   
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')
    

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능은 인간의 지능적인 행동을 컴퓨터 시스템에서 모방하는 기술입니다.

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 현재의 인공지능은 의식이 없습니다.



In [9]:
for idx, content in enumerate(chat_session.history):
    print(f'{content.__class__.__name__}{idx}]')
    print(content)

Content0]
parts {
  text: "인공지능에 대해 한 문장으로 짧게 설명하세요."
}
role: "user"

Content1]
parts {
  text: "인공지능은 인간의 지능적인 행동을 컴퓨터 시스템에서 모방하는 기술입니다.\n"
}
role: "model"

Content2]
parts {
  text: "의식이 있는지 한 문장으로 답하세요."
}
role: "user"

Content3]
parts {
  text: "현재의 인공지능은 의식이 없습니다.\n"
}
role: "model"



- 3번의 대화 턴이 발생했다면 첫 번째 턴부터 세 번째 턴까지 모두 합해 총 6번(메세지가 누적되는 구조, 1+2+3)의 변환 과정을 거쳐야 한다. 이런 과정 때문에 오버헤드가 발생한다.

### 2.3.2. Part 객체

![Content 클래스와 part 클래스](https://wikidocs.net/images/page/229769/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-02-08_105932.png)
- 여러 가지 형태의 데이터를 담는 구조로 api가 설계
    - 제미나이가 멀티모달 ai를 추구하기 때문
- 예시<br>
![이미지와 텍스트를 모델이 전달하면](https://wikidocs.net/images/page/229769/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-08-06_021020.png)


In [1]:
import google.generativeai as genai
import PIL.Image

image_data = PIL.Image.open("./images/monalisa.jpg") # 모나리자 그림
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(["이 그림에 대해 한 문장으로 설명하세요.", image_data])
print(response.text)


레오나르도 다 빈치가 그린 모나리자는 암갈색 옷을 입고 앉아 있는 여성의 초상화입니다.


## 2.4. 출력 구조 살펴보기

In [2]:
print(response._result)

candidates {
  content {
    parts {
      text: "레오나르도 다 빈치가 그린 모나리자는 암갈색 옷을 입고 앉아 있는 여성의 초상화입니다."
    }
    role: "model"
  }
  finish_reason: STOP
  avg_logprobs: -0.20041952906428157
}
usage_metadata {
  prompt_token_count: 269
  candidates_token_count: 37
  total_token_count: 306
}
model_version: "gemini-1.5-flash"



- candidates 필드와 usage_metadata  필드로 나뉨.
- 응답메시지 / 입출력 사용 토큰 수

### 2.4.1. Candidate 객체

In [3]:
print(f"건수: {len(response.candidates)}")
print("="*50)
for candidate in response.candidates:
    print(candidate)

건수: 1
content {
  parts {
    text: "레오나르도 다 빈치가 그린 모나리자는 암갈색 옷을 입고 앉아 있는 여성의 초상화입니다."
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.20041952906428157



- candidates 라는 이름에서 볼 수 있듯이 candidate가 여러개 들어가 있음.
- 웹 버전 제미나이에는 candidate를 3개 만듬.

### 2.4.2. FinishReason 객체

In [4]:
print(f"finish_reason: {response.candidates[0].finish_reason.name}, {response.candidates[0].finish_reason}")


finish_reason: STOP, 1


- stop(정상종료) / max_tokesn(최대 토큰 도달) / safety(안전성 문제) / recitation(텍스트 반복) / other(기타) 등의 응답이 있음.


### 2.4.3. Gemini 출력 구조
![출력 구조 다이어그램](https://wikidocs.net/images/page/229874/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-09-20_230856.png)